In [26]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, StratifiedKFold
from collections import Counter
%matplotlib inline

### 표본추출

#### 단순랜덤추출법

In [2]:
data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns=
                    [col.replace(' (cm)', '') for col in iris_cols])
print(iris.head(3))

   sepal length  sepal width  petal length  petal width  target
0           5.1          3.5           1.4          0.2     0.0
1           4.9          3.0           1.4          0.2     0.0
2           4.7          3.2           1.3          0.2     0.0


In [3]:
data_list = [1,2,3,4,5,'a','b','c']
random.sample(data_list, 4)

[4, 'b', 5, 2]

In [4]:
# 넘파이 라이브러리
np.random.choice(data_list, 4, replace=True)

array(['c', '1', 'b', 'b'], dtype='<U11')

In [5]:
# 특정 범위 내의 정수/실수의 난수 생성
print('0~10 사이의 정수 중 3개의 난수 생성: ', np.random.randint(0, 10, 3))
print('0~1 사이의 실수를 2*2 배열로 생성:\n', np.random.rand(2,2))

0~10 사이의 정수 중 3개의 난수 생성:  [3 8 3]
0~1 사이의 실수를 2*2 배열로 생성:
 [[0.20580088 0.39284645]
 [0.49833292 0.82017272]]


In [6]:
print(iris.sample(n=3, replace=False))

     sepal length  sepal width  petal length  petal width  target
126           6.2          2.8           4.8          1.8     2.0
60            5.0          2.0           3.5          1.0     1.0
119           6.0          2.2           5.0          1.5     2.0


In [7]:
# frac 파라미터로 전체 데이터의 3%를 랜덤으로 추출할 수 있음.
print(iris.sample(frac=0.03))

    sepal length  sepal width  petal length  petal width  target
9            4.9          3.1           1.5          0.1     0.0
79           5.7          2.6           3.5          1.0     1.0
61           5.9          3.0           4.2          1.5     1.0
70           5.9          3.2           4.8          1.8     1.0


In [8]:
print(iris.sample(frac=0.03, weights='sepal length'))

     sepal length  sepal width  petal length  petal width  target
129           7.2          3.0           5.8          1.6     2.0
23            5.1          3.3           1.7          0.5     0.0
149           5.9          3.0           5.1          1.8     2.0
4             5.0          3.6           1.4          0.2     0.0


In [9]:
print(iris.sample(3, axis=1).head(3))

   petal width  sepal length  sepal width
0          0.2           5.1          3.5
1          0.2           4.9          3.0
2          0.2           4.7          3.2


- 계통추출법

In [10]:
def sys_sampling(data, n) : 
    N = len(data)
    K = N//n
    index = data[:K].sample(1).index  # 첫 구간에서 임의로 선택한 샘플 1개의 인덱스
    # index개씩 띄어서 각 구간에서 하나씩 샘플 추출
    sys_df = pd.DataFrame()
    while len(sys_df) < n : 
        # sys_df = sys_df.append(data.loc[index,:])
        sys_df = pd.concat([sys_df, data.loc[index,:]])
        index += K
    return sys_df

print(sys_sampling(iris, 8))

     sepal length  sepal width  petal length  petal width  target
15            5.7          4.4           1.5          0.4     0.0
33            5.5          4.2           1.4          0.2     0.0
51            6.4          3.2           4.5          1.5     1.0
69            5.6          2.5           3.9          1.1     1.0
87            6.3          2.3           4.4          1.3     1.0
105           7.6          3.0           6.6          2.1     2.0
123           6.3          2.7           4.9          1.8     2.0
141           6.9          3.1           5.1          2.3     2.0


- 집락 추출법/층화추출법

In [11]:
def start_random_sampling(data, stratum, sampling_no, proportion=True) :
    if proportion == True : # 비례층화추출법: 원본 데이터 개수의 비율대로 추출
        levels = data[stratum].unique()
        total = data[stratum].value_counts().sum()
        prop_val = data[stratum].value_counts()/total
        no = prop_val * sampling_no
        result = pd.DataFrame()
        for level in levels : 
            temp_df = data[data[stratum]==level].sample(int(no[level]))
            result = pd.concat([result, temp_df])

    else :  # 불비례층화추출법: 임의로 정한 특정 비율대로 샘플링
        levels = list(proportion.keys())
        prop_val = np.array(list(proportion.values()))
        total = sum(prop_val)
        no = prop_val * sampling_no
        if total != 1 : 
            raise Exception('proportion sum is supposed to be 1.')
        else : 
            result = pd.DataFrame()
            for level in levels : 
                temp_df = data[data[stratum]==level].sample(int(no[level]))
                result = pd.concat([result, temp_df])

    return result

In [12]:
# 원본 층별 데이터 개수 확인: 층별로 동일하게 50개씩 관측값을 가지는 데이터
iris['target'].value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

In [13]:
# 비례층화추출법으로 9개 샘플링하기: 원본과 동일한 비율로 샘플링하기
print(start_random_sampling(iris, 'target', 9))

     sepal length  sepal width  petal length  petal width  target
5             5.4          3.9           1.7          0.4     0.0
6             4.6          3.4           1.4          0.3     0.0
38            4.4          3.0           1.3          0.2     0.0
61            5.9          3.0           4.2          1.5     1.0
70            5.9          3.2           4.8          1.8     1.0
95            5.7          3.0           4.2          1.2     1.0
115           6.4          3.2           5.3          2.3     2.0
134           6.1          2.6           5.6          1.4     2.0
139           6.9          3.1           5.4          2.1     2.0


In [14]:
# 불비례층화추출법으로 10개 샘플링하기: 임의로 정한 비율로 샘플링하기
print(start_random_sampling(iris, 'target', 10, proportion={0:0.2, 1:0.5,
                                                            2:0.3}))

     sepal length  sepal width  petal length  petal width  target
27            5.2          3.5           1.5          0.2     0.0
48            5.3          3.7           1.5          0.2     0.0
56            6.3          3.3           4.7          1.6     1.0
93            5.0          2.3           3.3          1.0     1.0
55            5.7          2.8           4.5          1.3     1.0
99            5.7          2.8           4.1          1.3     1.0
80            5.5          2.4           3.8          1.1     1.0
128           6.4          2.8           5.6          2.1     2.0
149           5.9          3.0           5.1          1.8     2.0
109           7.2          3.6           6.1          2.5     2.0


### 데이터 분할

- 일반적인 데이터 분할 및 홀드아웃 방법

In [16]:
X = iris.drop('target', axis=1)
y = iris.filter(['target'])

# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print('X_train(y_train): %d(%d), X_test(y_test): %d(%d)' % (len(X_train), len(y_train), len(X_test), len(y_test)))
print('X_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)), '\n')

# 홀드아웃 방법
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
print('X_train(y_train): %d(%d), X_test(y_test) %d(%d)' % (len(X_train), len(y_train), len(X_test), len(y_test)))
print('X_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))

X_train(y_train): 105(105), X_test(y_test): 45(45)
X_train의 비율: 0.70, X_test의 비율: 0.30 

X_train(y_train): 75(75), X_test(y_test) 75(75)
X_train의 비율: 0.50, X_test의 비율: 0.50


- Bootstrap: 랜덤복원추출

In [21]:
def Bootstrap(X, y, test_size = 0.3) : 
    train_size = 1 - test_size
    train_X = X.sample(frac=train_size)
    sampled_train_id = train_X.index
    sampled_test_id = X.drop(train_X.index, axis=0).index
    train_y = y.iloc[sampled_train_id,:]
    test_X = X.iloc[sampled_test_id,:]
    test_y = y.iloc[sampled_test_id,:]
    return train_X, test_X, train_y, test_y

for i in range(3) : 
    X_train, X_test, y_train, y_test = Bootstrap(X, y)
    print(f'Sample {i} ==> train_index: {X_train.index[:3]}, test_index: {X_test.index[:3]}')
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']), '\n')


Sample 0 ==> train_index: Int64Index([98, 19, 96], dtype='int64'), test_index: Int64Index([4, 8, 9], dtype='int64')
	X_train의 비율: 0.70, X_test의 비율: 0.30
	y_train의 타겟 구성: Counter({0.0: 40, 2.0: 35, 1.0: 30})
	y_test의 타겟 구성: Counter({1.0: 20, 2.0: 15, 0.0: 10}) 

Sample 1 ==> train_index: Int64Index([73, 111, 114], dtype='int64'), test_index: Int64Index([0, 4, 7], dtype='int64')
	X_train의 비율: 0.70, X_test의 비율: 0.30
	y_train의 타겟 구성: Counter({2.0: 37, 1.0: 35, 0.0: 33})
	y_test의 타겟 구성: Counter({0.0: 17, 1.0: 15, 2.0: 13}) 

Sample 2 ==> train_index: Int64Index([16, 97, 72], dtype='int64'), test_index: Int64Index([1, 2, 4], dtype='int64')
	X_train의 비율: 0.70, X_test의 비율: 0.30
	y_train의 타겟 구성: Counter({1.0: 38, 2.0: 35, 0.0: 32})
	y_test의 타겟 구성: Counter({0.0: 18, 2.0: 15, 1.0: 12}) 



- Shuffle Split: 무작위 순열 교차 검증

In [23]:
ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=3)
for i, (train_index, test_index) in enumerate(ss.split(X)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']), '\n')

Sample 0 ==> train_index: [146 106  14], test_index: [ 62 131 107]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({2.0: 26, 1.0: 26, 0.0: 23})
	y_test의 타겟 구성: Counter({0.0: 27, 1.0: 24, 2.0: 24}) 

Sample 1 ==> train_index: [111  87  48], test_index: [ 33 144   9]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({2.0: 31, 1.0: 24, 0.0: 20})
	y_test의 타겟 구성: Counter({0.0: 30, 1.0: 26, 2.0: 19}) 

Sample 2 ==> train_index: [ 10 124  62], test_index: [ 71 116  16]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({0.0: 26, 2.0: 26, 1.0: 23})
	y_test의 타겟 구성: Counter({1.0: 27, 2.0: 24, 0.0: 24}) 



- K-fold 분할

In [25]:
kf = KFold(n_splits=4, shuffle=False)
for i, (train_index, test_index) in enumerate(kf.split(X)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']), '\n')

Sample 0 ==> train_index: [38 39 40], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test의 타겟 구성: Counter({0.0: 38}) 

Sample 1 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 0.0: 12}) 

Sample 2 ==> train_index: [0 1 2], test_index: [76 77 78]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test의 타겟 구성: Counter({1.0: 24, 2.0: 13}) 

Sample 3 ==> train_index: [0 1 2], test_index: [113 114 115]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test의 타겟 구성: Counter({2.0: 37}) 



- Stratified K-fold 분할: 불균형 방지

In [27]:
skf = StratifiedKFold(n_splits=4)
# 분할 시 y를 고려해야 하기 때문에 split에 y를 입력해 주어야 함.
for i, (train_index, test_index) in enumerate(skf.split(X, y)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']))

Sample 0 ==> train_index: [13 14 15], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12})
Sample 1 ==> train_index: [0 1 2], test_index: [13 14 15]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12})
Sample 2 ==> train_index: [0 1 2], test_index: [26 27 28]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12})
Sample 3 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12})


- Group K-fold 분할